In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/교통사고 데이터 셋/구급출동_전주_2020_2022_최종.csv",encoding='cp949')
df.head()
df.dropna(subset=['위치정보X','위치정보Y'],inplace=True)
df = df.loc[df['환자연령']!=999,:]
df.reset_index(drop=True,inplace=True)

In [3]:
#사고발생장소
df['사고발생장소'].value_counts()

사고발생장소
도로            5571
도로외교통지역        594
집              343
기타              84
상업시설            37
오락/문화시설         19
학교/교육시설         18
바다/강/산/논밭       17
공장/산업/건설시설      13
의료관련시설          11
운동시설             7
집단거주시설           7
Name: count, dtype: int64

# 사고 발생 좌표 => 위도, 경도 변환

In [4]:
from pyproj import Proj, transform

proj_5181 = Proj(init='epsg:5181')
proj_4326 = Proj(init='epsg:4326')

def convert_5181_to_4326(x, y):
    # EPSG:5181에서 EPSG:4326으로 변환
    lon, lat = transform(proj_5181, proj_4326, x, y)
    return lat, lon


c:\Users\wjdtj\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\wjdtj\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
df['위도'],df['경도']=convert_5181_to_4326(df['위치정보X'], df['위치정보Y'])
df = df[df['경도']>126.85]

C:\Users\wjdtj\AppData\Local\Temp\ipykernel_27316\3218478162.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_5181, proj_4326, x, y)


In [99]:
df.loc[0:5,"위도":"경도"]

,위도,경도
0,35.877721,127.076733
1,35.845234,127.121687
2,35.806729,127.117422
3,35.837220,127.138885
4,35.832971,127.102032
5,35.847599,127.117781


# 버스 정류소 데이터 셋

In [100]:
df_bus=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/버스정류소/전주시 정류소 현황_20210426.csv",encoding='cp949')
df_bus.head()
df_bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   설치 지점명  569 non-null    object 
 1   지역(구)   569 non-null    object 
 2   지역(동)   534 non-null    object 
 3   세부주소    569 non-null    object 
 4   행정동     502 non-null    object 
 5   위도      569 non-null    float64
 6   경도      569 non-null    float64
dtypes: float64(2), object(5)
memory usage: 31.2+ KB


In [93]:
df_bus_loc=bus.loc[:,['경도','위도']]
df_bus_loc.head()
df_bus.to_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0524/bus_0524.csv")

# 버스정류장과 사고 장소 사이의 거리 계산

In [35]:
pip install numpy scipy pyproj

Note: you may need to restart the kernel to use updated packages.


In [103]:
import numpy as np
from scipy.spatial.distance import cdist
from pyproj import Transformer
from haversine import haversine

In [104]:
def find_nearest_bus(df):
    results = [] #결과를 담을 객체 생성

    for _, pedestrian in df[['위도','경도']].iterrows(): # 보행자의 위치별로 시행
        #가까운 학교와 최소 거리 초기화
        min_distance = float('inf')
        nearest_bus = None

        for _, bus in df_bus.iterrows(): #보행자별 각 정류장 시행
            bus_location = (bus['위도'], bus['경도'])
            pedestrian_location = (pedestrian['위도'], pedestrian['경도'])

            #거리계산
            distance = haversine(pedestrian_location, bus_location, unit='m') #거리 기준 변환가능
            
            if distance < min_distance: #최소거리와 정류장을 담음
                min_distance = distance
                nearest_bus = bus
        #보행자별 시행을 마치고 result에 값을 담음
        results.append({
                '보행자_위도': pedestrian['위도'],
                '보행자_경도': pedestrian['경도'],
                '가장_가까운_버스정류장': nearest_bus['설치 지점명'],  
                '가장_가까운_정류장_위도': nearest_bus['위도'],
                '가장_가까운_정류장_경도': nearest_bus['경도'],
                '버스정류장으로부터_거리': min_distance
            })
        
    return pd.DataFrame(results,index=df.index)

In [105]:
df2=find_nearest_bus(df)

In [106]:
df2.head()

,보행자_위도,보행자_경도,가장_가까운_버스정류장,가장_가까운_정류장_위도,가장_가까운_정류장_경도,버스정류장으로부터_거리
0,35.877721,127.076733,반월초교,35.876251,127.075730,186.788547
1,35.845234,127.121687,기린대로덕진공원,35.845746,127.121368,63.752142
2,35.806729,127.117422,안행교,35.806706,127.117581,14.569707
3,35.837220,127.138885,금암중앙하이츠,35.838063,127.137693,142.575979
4,35.832971,127.102032,서곡초교,35.832874,127.099705,210.031036


In [122]:
df2['버스_정류장_유무(10m)'] = df2['버스정류장으로부터_거리']<=10

In [123]:
df2.to_csv('C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0524/bus.csv')

In [109]:
df['가장_가까운_버스정류장']=df2['가장_가까운_버스정류장']
df['버스정류장으로부터_거리']=df2['버스정류장으로부터_거리']

In [114]:
df['버스정류장으로부터_거리'].describe()

count     6737.000000
mean       239.060520
std       1035.969945
min          1.485638
25%         75.468532
50%        138.170797
75%        243.227696
max      73223.311719
Name: 버스정류장으로부터_거리, dtype: float64

In [118]:
df['버스_정류장_유무'] = df['버스정류장으로부터_거리']<=10

In [119]:
df['버스_정류장_유무'].value_counts()

버스_정류장_유무
False    6678
True       59
Name: count, dtype: int64